In [ ]:
from collections import defaultdict


def parse_input(input_file):
    coords = []
    with open(input_file) as f:
        for line in f:
            line = line.rstrip()
            p1, p2 = line.split(",")
            coords.append((int(p1), int(p2)))
    return coords


def part1(input_file):
    ans = 0
    coords = parse_input(input_file)
    n = len(coords)
    for i in range(n):
        x1, y1 = coords[i]
        for j in range(i + 1, n):
            x2, y2 = coords[j]
            dx = abs(x1 - x2) + 1
            dy = abs(y1 - y2) + 1
            ans = max(ans, dx * dy)
    return ans


def part2(input_file):
    ans = 0
    coords = parse_input(input_file)
    by_x = defaultdict(list)
    by_y = defaultdict(list)
    for x, y in coords:
        by_x[x].append(y)
        by_y[y].append(x)
    hsegs, vsegs = [], []
    for x, ys in by_x.items():
        ys.sort()
        for i in range(len(ys) // 2):
            a, b = ys[2 * i], ys[2 * i + 1]
            a, b = min(a, b), max(a, b)
            hsegs.append((x, a, b))

    for y, xs in by_y.items():
        xs.sort()
        for i in range(len(xs) // 2):
            a, b = xs[2 * i], xs[2 * i + 1]
            a, b = min(a, b), max(a, b)
            vsegs.append((y, a, b))

    for i, a in enumerate(coords):
        for b in coords[:i]:
            minx = min(a[0], b[0])
            maxx = max(a[0], b[0])
            miny = min(a[1], b[1])
            maxy = max(a[1], b[1])
            # check cross any edge
            valid = True
            for hx, ha, hb in hsegs:
                if minx < hx < maxx:
                    valid = miny >= hb or maxy <= ha
                    if not valid:
                        break
            if not valid:
                continue
            for hy, ha, hb in vsegs:
                if miny < hy < maxy:
                    valid = minx >= hb or maxx <= ha
                    if not valid:
                        break
            if not valid:
                continue
            dx = abs(a[0] - b[0]) + 1
            dy = abs(a[1] - b[1]) + 1
            ans = max(ans, dx * dy)
    return ans


In [6]:
part1('input/day9_test.txt')

50

In [7]:
part1('input/day9.txt')

4771532800

In [24]:
part2('input/day9_test.txt')

24

In [25]:
part2('input/day9.txt')

1544362560

In [ ]:
import numpy as np

p = np.loadtxt("input/day9.txt", delimiter=",", dtype=int)

maxa = 0
maxb = 0
by_x = defaultdict(list)
by_y = defaultdict(list)
best_rect = None

for q in p:
    by_x[q[0]].append(q[1])
    by_y[q[1]].append(q[0])

h_segs = []
v_segs = []

for x, ys in by_x.items():
    yss = sorted(ys)
    for i in range(len(yss) // 2):
        a = [x, yss[2 * i]]
        b = [x, yss[2 * i + 1]]
        h_segs.append((a, b))

for y, xs in by_y.items():
    xss = sorted(xs)
    for i in range(len(xss) // 2):
        a = [xss[2 * i], y]
        b = [xss[2 * i + 1], y]
        v_segs.append((a, b))

for i, a in enumerate(p):
    for b in p[:i]:
        minx = min(a[0], b[0])
        maxx = max(a[0], b[0])

        miny = min(a[1], b[1])
        maxy = max(a[1], b[1])

        works = True
        for h0, h1 in h_segs:
            hx = h0[0]
            hy0 = min(h0[1], h1[1])
            hy1 = max(h0[1], h1[1])
            if hx > minx and hx < maxx:
                ok = hy1 <= miny or hy0 >= maxy
                if not ok:
                    works = False
                    break

        if works:
            for v0, v1 in v_segs:
                vy = v0[1]
                vx0 = min(v0[0], v1[0])
                vx1 = max(v0[0], v1[0])
                if vy > miny and vy < maxy:
                    ok = vx1 <= minx or vx0 >= maxx
                    if not ok:
                        works = False
                        break

        area = np.prod(np.abs(a - b) + 1)
        maxa = max(area, maxa)
        if works:
            if area > maxb:
                maxb = area
                best_rect = (
                    minx,
                    miny,
                    maxx,
                    maxy,
                )

print(maxa)
print(maxb)

if best_rect:
    bx0, by0, bx1, by1 = best_rect
    bw = abs(bx1 - bx0) + 1
    bh = abs(by1 - by0) + 1
    extras = np.array([[bx0, bx1 + 1], [by0, by1 + 1]])
else:
    bx0 = by0 = bw = bh = 0
    extras = np.empty((2, 0), dtype=int)

coords = np.concatenate([p.T, extras], axis=1)
x_min = np.min(coords[0])
x_max = np.max(coords[0])
y_min = np.min(coords[1])
y_max = np.max(coords[1])

width = max(1, x_max - x_min + 1)
height = max(1, y_max - y_min + 1)

scale = 100
lines = []
for a, b in h_segs:
    lines.append(
        f'<line x1="{a[0] / scale}" y1="{a[1] / scale}" x2="{b[0] / scale}" y2="{b[1] / scale}" stroke="black" stroke-width="1" stroke-linecap="square" />'
    )
for a, b in v_segs:
    lines.append(
        f'<line x1="{a[0] / scale}" y1="{a[1] / scale}" x2="{b[0] / scale}" y2="{b[1] / scale}" stroke="black" stroke-width="1" stroke-linecap="square" />'
    )
rect = ""
if best_rect:
    rect = f'<rect x="{bx0 / scale}" y="{by0 / scale}" width="{bw / scale}" height="{bh / scale}" fill="#790096" />'

svg = [
    f'<svg xmlns="http://www.w3.org/2000/svg" width="{width // scale + 1}" height="{height // scale + 1}" viewBox="{x_min // scale} {y_min // scale} {width // scale + 1} {height // scale + 1}">',
    f'<rect x="{x_min // scale}" y="{y_min // scale}" width="{width // scale + 1}" height="{height // scale + 1}" fill="white" />',
    rect,
    *lines,
    "</svg>",
]

with open("09.svg", "w") as f:
    f.write("\n".join(filter(None, svg)))

4771532800
1544362560
